Let's do 3+1 dimensions, consider $x^\mu \in \mathbb{R}^{1+3}$, where $f$ is the Fourier transform of a Gaussian:

\begin{align}
  [\phi(x), \phi(0)] = \int_{-\infty}^\infty \frac{\mathrm d^3p}{(2\pi)^3} \frac{f^2(|\vec p|)}{2 F_{\vec p}} \left( e^{-ip_\mu x^\mu} - e^{ip_\mu x^\mu}\right) \bigg\rvert_{p^0 = F_{\vec p}} \\
\end{align}

Choose spherical coordinates $\vec p = (r, \theta, \phi)$ with $\hat z$ direction along $\vec x$. Now let $t = x^0$ and $x = |\vec x|$. Assuming $m \neq 0$ and $\sigma \neq 0$, the integral simplifies to:

\begin{align}
  [\phi(x), \phi(0)] = -i \int_{0}^\infty \frac{\mathrm dr}{2\pi^2} t r^2 \, \mathrm{sinc}(rx) \, f^2(r^2) \, \mathrm{sinc}(t E_r f(r^2))
\end{align}

Rescale $r \mapsto r' = r/\sigma$ and instead of plotting a $t$ vs. $x$ diagram, plot the dimensionless quantities $t' = mt$ vs. $x' = mx$. Then the only parameter of this commutator is $\alpha = \sigma m$ with units of energy, up to an overall factor of $\sigma^{-2}$.

\begin{align}
  \left| [\phi(x), \phi(0)] \right| &= \frac{1}{\sigma^2} \cdot \frac{t'}{2\pi^2 \alpha} \int_0^\infty \mathrm dr \, r^2 \, e^{-r^2} \, \mathrm{sinc}\left(\frac{rx'}{\alpha}\right) \, \mathrm{sinc}\left(t' e^{-\frac12 r^2} \sqrt{\frac{r^2}{\alpha^2} + 1}\right)
\end{align}

In [ ]:
NUM_PROC = 56
FOLDER = "./output/"

from IPython.utils import io
import numpy as np
import multiprocessing as mp

In [ ]:
%%writefile temp.py

import numpy as np
import scipy.integrate as integrate
import warnings

def integrand(r, x, t, a):
    f = np.exp(-1/2 * r**2)
    return r**2 * f**2 * np.sinc(r*x/a) * np.sinc(t * f * np.sqrt(r**2/a**2 + 1))
    
def comm(x, t, a):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        I = integrate.quad(integrand, 0, +np.inf, args=(x, t, a))[0]    
    
    return abs(1/(2 * np.pi**2 * a) * t * I)**2

In [ ]:
from temp import comm

def calc(lim_x, lim_t, step, alpha):
    xs = np.linspace(-lim_x, lim_x, step)
    ts = np.linspace(-lim_t, lim_t, step)
    
    xx, tt = np.meshgrid(xs, ts)
    arguments = [(x, t, alpha) for t in ts for x in xs]

    mp_context = mp.get_context("spawn")
    with mp_context.Pool(processes=NUM_PROC) as pool:
        z = pool.starmap(comm, arguments)

    z = np.array(z).reshape(step, step)
    return xx, tt, z

In [ ]:
import matplotlib.pyplot as plt

def plot(xx, tt, z, sigma, mass, label=None, log=False, show=True):
    ax = plt.gca()
    ax.axis('equal')
    ax.set_box_aspect(1)
    
    cnt = plt.pcolormesh(xx, tt, z, cmap='viridis', antialiased=False, shading='auto')
    
    plt.colorbar()
    plt.xlabel("Space Coordinate $m x$")
    plt.ylabel("Time Coordinate $m t$")
    plt.title("$" + (r"\log" if log else "") + "|\sigma^2 [\phi(mt,0,0,mx), \phi(0,0,0,0)]|^2$, "
                  + f"$\sigma = {sigma}, m = {mass}$")
    
    if label:
        plt.savefig(FOLDER + label + ("-log" if log else "") + ".png")
    
    if show:
        plt.show()
    
def run1(lim_x, lim_t, step, sigma, mass, label=None, log=False, show=True):
    alpha = sigma * mass
    
    xx, tt, z = calc(lim_x, lim_t, step, alpha)
    
    if not label:
        label = f"numerical-commutator-m{mass}-s{sigma}-l{lim_x}-{lim_t}"
    
    plot(xx, tt, z, sigma, mass, label=label, show=show)
    
    if log:
        plot(xx, tt, np.log(z), sigma, mass, label=label, log=True, show=show)

In [ ]:
from tqdm.notebook import tqdm
for s in tqdm([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,
          0.8, 0.9 , 1,1.1, 1.2, 1.4, 1.5, 1.6, 1.8, 2,
          3,4, 5,6, 7,8, 9, 10]):
    run1(20, 20, 250, s, 1, log=True)